# **Age Detection with OpenCV**

![대체 텍스트](https://www.pyimagesearch.com/wp-content/uploads/2020/04/opencv_age_detection_examples.jpg)

sypically, you’ll see age detection implemented as a two-stage process:



1.   Detect faces in the input image/video stream
2.   Extract the face Region of Interest (ROI), and apply the age detector algorithm to predict the age of the person

Age detector modle

![대체 텍스트](https://www.pyimagesearch.com/wp-content/uploads/2020/04/opencv_age_detection_arch.png)

In [ ]:
#@title Import Libraries
# import the necessary packages
import os
import numpy as np
import cv2
import google.colab.patches

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
import six.moves.urllib.request 
import six 
import PIL

import base64, logging
import io
import IPython.display
import google.colab.output

Download the pre-trained detection models, consisting of under files:
>
* 1.Face Detection Model
* 2.Age Detection Model
* 3.Gender Detection Model

In [ ]:
#@title Load data and model
%%shell
mkdir ageDetection; cd ageDetection
mkdir model; cd model
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/face_deploy.prototxt 
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/face_deploy.caffemodel

curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/age_deploy.prototxt
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/age_deploy.caffemodel

curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/gender_deploy.prototxt
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/gender_deploy.caffemodel

cd ..; mkdir images; cd images
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/jangnara.jpg 
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/jangnara2.jpg 

cd ..;mkdir videos; cd videos
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Videos/JJANG.mp4
cd videos
mkdir save

In [ ]:
# Root directory 
ROOT_DIR = os.path.abspath("./ageDetection/")
# Model directory 
MODEL_DIR = os.path.join(ROOT_DIR, "model")
# Images directory
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

# Video directory
VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "save")

In [ ]:
#Face
prototxtPath = os.path.sep.join([MODEL_DIR, "face_deploy.prototxt"])  
weightsPath = os.path.sep.join([MODEL_DIR, "face_deploy.caffemodel"]) 
faceNet = cv2.dnn.readNetFromCaffe(prototxtPath, weightsPath)
# faceNet= cv2.dnn.readNet(weightsPath, prototxtPath,"caffe")

#Age
prototxtPath = os.path.sep.join([MODEL_DIR, "age_deploy.prototxt"])  
weightsPath = os.path.sep.join([MODEL_DIR, "age_deploy.caffemodel"]) 
ageNet = cv2.dnn.readNetFromCaffe(prototxtPath, weightsPath)
# ageNet= cv2.dnn.readNet(weightsPath, prototxtPath,"caffe")

#Gender
prototxtPath = os.path.sep.join([MODEL_DIR, "gender_deploy.prototxt"])  
weightsPath = os.path.sep.join([MODEL_DIR, "gender_deploy.caffemodel"]) 
genderNet = cv2.dnn.readNetFromCaffe(prototxtPath, weightsPath)
# genderNet= cv2.dnn.readNet(weightsPath, prototxtPath,"caffe")

In [ ]:
#Age and Gender classification list
AGE_LIST = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]
GENDER_LIST = ['Male', 'Female']

# **1**.Detection model demo by picture

In [ ]:
#@title Select an Input Image
SAMPLE_IMAGE = 'jangnara2.jpg' #@param {type:"string"}
IMAGE_URL = 'https://www.bollywoodhungama.com/wp-content/uploads/2020/06/BTS-members-look-heavenly-in-head-to-toe-Prada-collection-on-the-August-cover-of-Vogue-Japan-.jpg'  #@param {type:"string"}

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

def download_and_resize_image(url,display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = six.moves.urllib.request.urlopen(url)
  image_data = response.read()
  image_data = six.BytesIO(image_data)
  pil_image = PIL.Image.open(image_data)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print(f"Image downloaded to {filename}.")
  if display:
    display_image(pil_image)
  return filename

if len(IMAGE_URL) !=0 :
  IMAGE_URL = download_and_resize_image(IMAGE_URL)
if len(SAMPLE_IMAGE) !=0 :
  SAMPLE_URL = os.path.join(IMAGE_DIR, SAMPLE_IMAGE)

image_url = IMAGE_URL or SAMPLE_URL

## Apply Detection model

In [ ]:
def detect_and_predict_age(frame,conf_threshold=0.5):
  (high, width) = frame.shape[:2]

  blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),(104.0, 177.0, 123.0))		
  faceNet.setInput(blob)
  detections = faceNet.forward()

  for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > conf_threshold:

			# extract the ROI of the face
      box = detections[0, 0, i, 3:7] * np.array([width, high, width, high])
      (startX, startY, endX, endY) = box.astype("int")			
      face = frame[startY:endY, startX:endX]			
      if face.shape[0] < 20 or face.shape[1] < 20:
        continue

      #Age prediction and display
      faceBlob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),(78.4263377603, 87.7689143744, 114.895847746),swapRB=False)
      ageNet.setInput(faceBlob)
      preds = ageNet.forward()
      arg = preds[0].argmax()
      age = AGE_LIST[arg]
      ageConfidence = preds[0][arg]
      ageText = f"{age}: {ageConfidence * 100:.2f}%"

			#Gender prediction and display
      genderNet.setInput(faceBlob)
      preds = genderNet.forward()
      arg = preds[0].argmax()
      gender = GENDER_LIST[arg]
      genderConfidence = preds[0][arg]
      genderText = f"{gender}: {genderConfidence * 100:.2f}%"

			# draw the bounding box of the face along with the associated probability
      y = startY - 30 if startY - 30 > 30 else startY + 30
      cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
      cv2.putText(frame, genderText, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
      cv2.putText(frame, ageText, (startX, y+15),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

In [ ]:
image = cv2.imread(image_url)
detect_and_predict_age(image)
google.colab.patches.cv2_imshow(image)

# **2**.Detection model demo by video

In [ ]:
#@title Videdo Capture
# Using a webcam to capture images for processing on the runtime.
# Source: https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=2viqYx97hPMi

def data_uri_to_img(uri):
  try:
    image = base64.b64decode(uri.split(',')[1], validate=True)
    # make the binary image, a PIL image
    image = PIL.Image.open(io.BytesIO(image))
    # convert to numpy array
    image = np.array(image, dtype=np.uint8);
    return image
  except Exception as e:
    logging.exception(e)
    print('\n')
    return None

def video_to_data_url(filename):
    ext = filename.split('.')[-1]
    prefix = f"data:video/{ext};base64,"
    with open(filename, 'rb') as f:
        vidoe = f.read()
    return prefix + base64.b64encode(vidoe).decode()

# from base64 import b64decode
# playing webcam or video with javascript
def take_photo(filename='photo.jpg', quality=0.8):
  js = IPython.display.Javascript('''     
    async function takePhoto(filename, quality) {
                  
      const div = document.createElement('div');
      document.body.appendChild(div);

      const exit = document.createElement('button');
      exit.textContent = 'Exit';
      div.appendChild(exit);

      const video = document.createElement('video');
      video.style.display = 'block';
           
      if('photo.jpg' == filename){
        const stream = await navigator.mediaDevices.getUserMedia({video: true}); 
        video.srcObject = stream;   
      }else{
        video.src = filename;
        video.type="video/mp4"
      }
      await video.play();  
      div.appendChild(video);       
                   

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      let jsLog = function(abc) {
        document.querySelector("#output-area").appendChild(document.createTextNode(`${abc}... `));
      }

      // when Exit button is clicked.   
      var isOpened = true; 
      var exitPromise = new Promise((resolve) => {exit.onclick = resolve});   
      exitPromise.then(()=>{isOpened = false; stream.getVideoTracks()[0].stop();});
      
      //when end of video
      var endPromise = new Promise((resolve) => {video.onended = resolve});   
      endPromise.then(()=>{isOpened = false; video.stop();});

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;

      for (let i = 0; isOpened; i++) {
        canvas.getContext("2d").clearRect(0, 0, canvas.width, canvas.height);
        canvas.getContext('2d').drawImage(video, 0, 0);
        img = canvas.toDataURL('image/jpeg', quality);

        // jsLog(i + "sending");
        // Call a python function and send this image
        google.colab.kernel.invokeFunction('notebook.run_ageDetection', [img], {});
        // jsLog(i + "SENT");

        // wait for X miliseconds second, before next capture
        await new Promise(resolve => setTimeout(resolve, 250));        
      }       
      div.remove();      
    }    
    ''')  
  # make the provided HTML, part of the cell
  IPython.display.display(js)
  #call the takePhoto() JavaScript function
  google.colab.output.eval_js(f"takePhoto('{filename}',{quality})")

frame_count = 0
writer = None
# InvokeFunction
# takes the numpy image and runs detection, then shows the results by visualizing
def run_ageDetection(uri): 
  global frame_count, writer

  image = data_uri_to_img(uri)     
  if writer is None:		
      fourcc = cv2.VideoWriter_fourcc(*'DIVX')  
      writer = cv2.VideoWriter(outVideo, fourcc, 2, (image.shape[1], image.shape[0]), True)
  try:        
    detect_and_predict_age(image)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    frame_count+=1    
    name = f"{frame_count}.jpg"
    name = os.path.join(VIDEO_SAVE_DIR, name)
    cv2.imwrite(name, image)

    if writer is not None:
      writer.write(image)
  except Exception as e:
    logging.exception(e)
    print('\n')

# register this function, so JS code could call this
google.colab.output.register_callback('notebook.run_ageDetection', run_ageDetection)

## Apply Detection model

In [ ]:
%%shell
cd ageDetection
rm ./videos/save/* 
rm ./videos/out.avi

In [ ]:
inVideo = os.path.join(VIDEO_DIR, "JJANG.mp4")
outVideo= os.path.join(VIDEO_DIR, "out.avi")

In [ ]:
frame_count = 0
# data_url = video_to_data_url(inVideo)
data_url = 'photo.jpg'
try: 
  # put the JS code in cell and run it
  take_photo(data_url)  
  if writer is not None:
    writer.release()   
except Exception as e:
  logging.exception(e)
  print('\n')

writer = None

<IPython.core.display.Javascript object>

## Downlod  video


In [ ]:
from google.colab import files
files.download(outVideo)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>